In [ ]:
from skimage.metrics import structural_similarity
import cv2
import imutils
import requests
from PIL import Image


In [ ]:
!mkdir ghscard_tamper
!mkdir ghscard_tamper/images

In [ ]:
#open images and display, both images show an orginal and a tampered one
original = Image.open(requests.get('https://theghanatech.com/wp-content/uploads/2021/06/Ghana-Card-original-min.jpg', stream=True).raw)
tampered = Image.open(requests.get('https://scontent.facc7-1.fna.fbcdn.net/v/t1.6435-9/35433362_2206549336039159_998699021532921856_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=tSu2jq4kvdYAX8ZQlXB&_nc_ht=scontent.facc7-1.fna&oh=00_AfCS7U88TL7h-p-hGmbD5G_nvfMRvJqqTpcEk_OkVSeMiw&oe=64CC9376', stream=True).raw)

In [ ]:
print("Original Image format: ", original.format)
print("Tampered Image format: ", tampered.format)

print("Original Image size: ", original.size)
print("Tampered Image size: ", tampered.size)

In [ ]:
#image resize into same sizes
original = original.resize((400, 400))
print (original.resize)
original.save('ghscard_tamper/images/original.jpg') #saves image into folder
tampered = tampered.resize((400, 400))
print (tampered.resize)
tampered.save('ghscard_tamper/images/tampered.jpg') #saves image into folder



In [ ]:
#display orginal image
original

In [ ]:
#display tampered image
tampered

In [ ]:
#upload/load the images into cv2
original = cv2.imread('ghscard_tamper/images/original.jpg')
tampered = cv2.imread('ghscard_tamper/images/tampered.jpg')

In [ ]:
#convert images to greyscale
original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
tampered = cv2.cvtColor(tampered, cv2.COLOR_BGR2GRAY)

In [ ]:
#compute structural similarity index (SSIM) between the two images, ensuring that the difference image is returned
(score, diff) = structural_similarity(original, tampered, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))

In [ ]:
#calculate threshold and contours
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)

In [ ]:
#loop over the contours
for c in contours:
    #apply contours to image
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 255), 2) # type: ignore
    cv2.rectangle(tampered, (x, y), (x + w, y + h), (0, 0, 255), 2) # type: ignore


In [ ]:
#display orginal image with contour
print("Original format Image")
Image.fromarray(original)

In [ ]:
#display tampered image with contour
print("Tampered format Image")
Image.fromarray(tampered)

In [ ]:
#display difference in Image with black 
print("Difference Image")
Image.fromarray(diff)

In [ ]:
#display threshold in Image with white
print("Threshold Image")
Image.fromarray(thresh)